In [ ]:
import pandas as pd
import numpy as np
from numpy import loadtxt
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

#PS: Pressure, bar, 100 Hz --> 100 Messungen pro Sekunde
#EPS: Motor power, W, 100 Hz 
#FS: Volume flow, l/min, 10 Hz --> 10 Messungen pro Sekunde
#TS: Temperature, Celsius, 1 Hz --> 1 Messung pro Sekunde
#VS: Vibration, mm/s, 1Hz
#CE: Cooling efficiency (virtual), %, 1 Hz
#CP: Cooling power (virtual), kW, 1 Hz
#SE: Efficency factor, %, 1 Hz

class DataProcessor:
    def __init__(self, input_path, file_names):
        self.input_path = input_path
        self.file_names = file_names
        
    def read_files(self):
        self.data = {}
        print("Reading files...")
        for file in self.file_names:
            with open(self.input_path + file + '.txt', 'r') as f:
                self.data[file] = loadtxt(f)
        return self.data
    
    def print_shape(self):
        print("Files read:")
        for file in self.data:
            print(f"{file}: {self.data[file].shape}")
            
    def create_target_df(self):
        target_columns = ['Cooler_Condition', 'Valve_Condition', 
                        'Internal_Pump_Leakage', 'Hydraulic_Accumulator', 
                        'Stable_Flag']
        self.df_target = pd.DataFrame(self.data['target'], columns=target_columns)
        self.valve_condition = self.df_target['Valve_Condition']
        del self.data['target']
        return self.valve_condition

def process_data():
    input_path = "input_data/"
    file_names = [
        "ce", "cp", "eps1", "se", "vs1", 
        "fs1", "fs2", 
        "ps1", "ps2", "ps3", "ps4", "ps5", "ps6",
        "ts1", "ts2", "ts3", "ts4", "target"
    ]
    
    processor = DataProcessor(input_path, file_names)
    data = processor.read_files()
    processor.print_shape()
    processor.create_target_df()
    df_target = processor.valve_condition
    return data, df_target

data, df_target = process_data()

In [2]:
dfs = {}


input_path = "input_data/"
file_names = [
    "ce", "cp", "eps1", "se", "vs1", 
    "fs1", "fs2", 
    "ps1", "ps2", "ps3", "ps4", "ps5", "ps6",
    "ts1", "ts2", "ts3", "ts4", "target"
]

for file in file_names:
    with open(input_path + file + '.txt', 'r') as f:
        data = pd.read_csv(f, sep="\t", header=None)
        dfs[file] = data

In [3]:
dfs['target'] = dfs['target'][1]

In [4]:
target = dfs['target']

In [5]:
dfs.pop("target")

0       100
1       100
2       100
3       100
4       100
       ... 
2200    100
2201    100
2202    100
2203    100
2204    100
Name: 1, Length: 2205, dtype: int64

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq

# Beispiel: Laden von simulierten Daten (Daten sollten entsprechend ersetzt werden)
data = {
    "PS1": np.sin(2 * np.pi * np.linspace(0, 1, 6000) * 10) + 0.5 * np.random.randn(6000),
    "PS2": np.sin(2 * np.pi * np.linspace(0, 1, 6000) * 20) + 0.5 * np.random.randn(6000),
    "EPS1": np.cos(2 * np.pi * np.linspace(0, 1, 6000) * 5) + 0.3 * np.random.randn(6000),
}

df = pd.DataFrame(data)

# Samplingrate und Dauer
fs = 100  # 100 Hz (Sampling-Rate)
n = df.shape[0]  # Anzahl der Abtastungen
t = np.linspace(0, n / fs, n, endpoint=False)  # Zeitvektor

# FFT für jede Spalte berechnen
fft_features = {}
for sensor, values in df.items():
    # FFT anwenden
    fft_result = fft(values)
    freq = fftfreq(n, d=1/fs)  # Frequenzen berechnen
    
    # Nur positive Frequenzen verwenden
    pos_freq_idx = freq > 0
    fft_result = np.abs(fft_result[pos_freq_idx])  # Amplitude
    freq = freq[pos_freq_idx]
    
    # Frequenzmerkmale extrahieren (z. B. Peaks, Gesamtenergie)
    fft_features[f"{sensor}_max_amp_freq"] = freq[np.argmax(fft_result)]
    fft_features[f"{sensor}_energy"] = np.sum(fft_result ** 2)
    fft_features[f"{sensor}_mean_amp"] = np.mean(fft_result)
    
    # Optional: Frequenzspektrum plotten
    plt.plot(freq, fft_result, label=sensor)
    
# Ergebnisse plotten
plt.title("Frequenzspektren der Sensoren")
plt.xlabel("Frequenz (Hz)")
plt.ylabel("Amplitude")
plt.legend()
plt.show()

# FFT-Merkmale als DataFrame speichern
fft_features_df = pd.DataFrame([fft_features])
print(fft_features_df)
